In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Purpose: To test the neuron decomposition pipeline using 
the vdi for microns that accesses the data through the caveclient
"""

'\nPurpose: To test the neuron decomposition pipeline using \nthe vdi for microns that accesses the data through the caveclient\n'

# Setting up the Cave Client

In [3]:
from caveclient import CAVEclient
client = CAVEclient()

In [5]:
CAVE_TOKEN  = "INSERT TOKEN HERE"

#this token is an example token but is not valid

CAVE_TOKEN="ba0a1b546e9b87f402c5d8c1d1d45bca" 
CAVE_TOKEN="ba0a1b546e9b87f402c5d8c1d1d45bca"

In [6]:
# Saving the token to caveclient package and as a local env file
try:
    client.auth.save_token(token=CAVE_TOKEN,overwrite=True)
except Exception as e:
    print(e)

filepath ="./.env" 
with open(filepath,"w+") as f:
    f.write(f"CAVE_TOKEN={CAVE_TOKEN}")

# Importing the VDI with microns_cave interface


In [7]:
from neurd.cave_interface import CaveInterface
from pathlib import Path

client = CaveInterface(
    "minnie65_public",
    env_filepath=Path(filepath).absolute()
)
client

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Accessing the data with the Cave Interface (we are just checking to make sure cave client access is working, lines of code not necessary for VDI to function)

In [8]:
client.materialize.tables


KeyboardInterrupt



In [ ]:
nucleus_seg_ids = client.segment_ids_with_nucleus()
nucleus_seg_ids[:10]

In [ ]:
# finding a segement id from the nuclues id
nucleus_id = 197805

nuc_df = client.neuron_nucleus_df()
seg_id = nuc_df.query(
    f"id_ref=={nucleus_id}")[
    "pt_root_id"].to_list()[0]
seg_id

In [ ]:
# downloading the mesh
mesh = client.mesh_from_seg_id(
    seg_id
)

In [ ]:
# downloading the synapses
syn_df = client.pre_post_df_from_seg_id(seg_id)
syn_df

# Using the VDI Microns Cave

In [ ]:
# assumes your caveclient token is stored in an env file
from neurd.vdi_microns_cave import volume_data_interface as vdi

In [ ]:
segment_id = 864691135502491741
syn_df = vdi.segment_id_to_synapse_df(segment_id)
syn_df

In [ ]:
syn_dict = vdi.segment_id_to_synapse_dict(segment_id)

In [ ]:
syn_dict['presyn']["synapse_coordinates"]

In [ ]:
vdi.client.segment_ids_with_nucleus()

In [ ]:
vdi.voxel_to_nm_scaling

# Neuron Preprocessing Pipeline

In [ ]:
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz

In [ ]:
segment_id = 864691135502491741

# Step 0: Loading Mesh

In [ ]:
mesh = vdi.fetch_segment_id_mesh(
    segment_id,
    plot = False
)
mesh

In [ ]:
from datasci_tools import pipeline

products = pipeline.PipelineProducts()
products

# Step 0.1 (optional): Visualizing Synapses with Mesh

In [ ]:
syn_dict = vdi.segment_id_to_synapse_dict(segment_id)

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    mesh,
    scatters = [
        syn_dict["postsyn"]["synapse_coordinates"],
        syn_dict["presyn"]["synapse_coordinates"]
    ],
    scatters_colors=["blue","yellow"]
)

# Step 1: Decimation

In [ ]:
decimation_parameters = dict(
    decimation_ratio =0.25,
)

In [ ]:
mesh_decimated = tu.decimate(
    mesh,
    **decimation_parameters
)

In [ ]:
ipvu.plot_objects(mesh_decimated)

In [ ]:
# saving any parameters used or output products for later use
products.set_stage_attrs(
    stage = "decimation",
    attr_dict = dict(
        decimation_parameters = decimation_parameters,
        segment_id = segment_id,
    ),
    
)

print(products)

# Step 2: Soma Identification

In [ ]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

In [ ]:
# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

In [ ]:
# overwritting decimated mesh because original was already decimated
mesh_decimated = mesh

In [ ]:
from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

In [ ]:
sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

In [ ]:
soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

In [ ]:
products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Step 2b: Saving off pipeline products

In [ ]:
from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

# Step 3: Decomposition

In [ ]:
from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

In [ ]:
nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

In [ ]:
decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

In [ ]:
print(neuron_obj.pipeline_products)

# Saving off neuron

In [ ]:
vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

In [ ]:
neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

In [ ]:
print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

In [ ]:
from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

In [ ]:
multi_soma_split_parameters = dict()

In [ ]:
_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


In [ ]:
neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

In [ ]:
neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

In [ ]:
n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

In [ ]:
from neurd import neuron_pipeline_utils as npu

In [ ]:
neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

In [ ]:
nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

In [ ]:
neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

In [ ]:
_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

In [ ]:
vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

In [ ]:
from neurd import synapse_utils as syu

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

In [ ]:
syu.plot_synapses(neuron_obj_proof)

In [ ]:
syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

In [ ]:
syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

In [ ]:
nviz.plot_compartments(neuron_obj_proof)

In [ ]:
from neurd import soma_extraction_utils as sm
soma_extraction_parameters = dict()

# could inspect the parameters of a certain category
from neurd import parameter_utils as paru
paru.category_param_from_module(
        module = sm,
        category = "glia",
)

# overwritting decimated mesh because original was already decimated
mesh_decimated = mesh

from neurd import neuron_visualizations as nviz

verbose = True

soma_products = sm.soma_indentification(
    mesh_decimated,
    verbose=verbose,
    **soma_extraction_parameters
)

sm.plot_soma_products(
    mesh_decimated,
    soma_products = soma_products,
    verbose = True
)

soma_products.soma_extraction_parameters = soma_extraction_parameters
soma_products

products.set_stage_attrs(
    stage = "soma_identification",
    attr_dict = soma_products,
)

print(products)

# Step 2b: Saving off pipeline products

from datasci_tools import system_utils as su
su.save_object(
    products,"products_up_to_soma_stage"
)

# Step 3: Decomposition

from neurd import neuron

neuron_obj = neuron.Neuron(
    mesh = mesh,
    segment_id = segment_id, # don't need this explicitely if segment_id is already in products
    pipeline_products = products,
    suppress_preprocessing_print=False,
    suppress_output=False,
)

neuron_obj

nviz.visualize_neuron(
    neuron_obj,
    limb_branch_dict="all"
)

# Calculate statistics

decomp_products = neuron_obj.calculate_decomposition_products(
    store_in_obj = True,
)

decomp_products

print(neuron_obj.pipeline_products)

# Saving off neuron

vdi.save_neuron_obj(
    neuron_obj,
    verbose = True
)

# Reloading the Neuron

neuron_obj_rec = vdi.load_neuron_obj(
    segment_id = segment_id,
    #mesh_decimated = mesh_decimated
)

print(neuron_obj_rec.pipeline_products)

# Step 4A: Multi Soma Split Suggestions

from neurd import neuron_visualizations as nviz
nviz.plot_soma_limb_concept_network(
    neuron_obj
)

multi_soma_split_parameters = dict()

_ = neuron_obj.calculate_multi_soma_split_suggestions(
    plot = True,
    store_in_obj = True,
    **multi_soma_split_parameters
)


neuron_obj.pipeline_products.multi_soma_split_suggestions.multi_soma_split_parameters = multi_soma_split_parameters

# Step 4B: Multi Soma Split Execution

neuron_list = neuron_obj.multi_soma_split_execution(
    verbose = False,
)

n1 = neuron_list[0]
print(n1.pipeline_products)

# Step 5: Cell Typing

from neurd import neuron_pipeline_utils as npu

neuron_obj_axon = npu.cell_type_ax_dendr_stage(
    n1,
    mesh_decimated = mesh_decimated,
    plot_axon = False,
)

nviz.plot_axon(
    neuron_obj_axon
)

# Autoproofreading Stage

neuron_obj_proof = npu.auto_proof_stage(
    neuron_obj_axon,
    mesh_decimated = mesh_decimated,
    calculate_after_proof_stats = False,
)

_ = npu.after_auto_proof_stats(
    neuron_obj_proof,
    store_in_obj = True,
)

# Saving off the autoproofread neuron

vdi.save_neuron_obj_auto_proof(
    neuron_obj_proof,
)

# Plotting the final neuron

from neurd import synapse_utils as syu

from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    neuron_obj_axon.mesh,
    meshes = [neuron_obj_proof.mesh],
    meshes_colors="red"
)

syu.plot_synapses(neuron_obj_proof)

syu.plot_synapses_error_from_neuron_obj(neuron_obj_proof)    

syu.plot_synapses_valid_from_neuron_obj(neuron_obj_proof)  

nviz.plot_compartments(neuron_obj_proof)